# Yeast cells detection sample notebook

In [ ]:
import sys
sys.path.append('../')
#sys.path.append('%HOMEPATH%/detectron2') #insert local path to detecron2
import numpy as np

import yeastcells.clustering as clustering
import yeastcells.features as features
import yeastcells.data as data
import yeastcells.model as model
import yeastcells.output as output
import yeastcells.fp as fp
import yeastcells.visualize as visualize

In [ ]:
cd ~

### Load data

In [ ]:
path = '../../slow-data/yeastcells/data/DATA_CIT/Movie_1/tiffs'
filenames = data.load_data(path)
print(filenames[0])

In [ ]:
fn = filenames[0]
image = data.read_image(fn)
np.shape(image) #(frames, length, width, channels)

### Load and run model on data for segmentation

In [ ]:
#set path to model_final.pth
model_path = '/var/tensorflow-logs/p253591/yeast-cell-detection-run-4'

#load model
predictor = model.load_model(model_path, seg_thresh=0.94, device='cuda:0') #set GPU if available otherwise use 'cpu'

In [ ]:
output = [
  {'instances': predictor(frame)['instances'].to('cpu')}
  for frame in image
]

### Run clustering algorithm for tracking

In [ ]:
labels, coordinates = clustering.cluster_cells(output, dmax=5, min_samples=5, eps=0.4, progress=False)

In [ ]:
#plot paths
ax = visualize.plot_paths(
  labels,
  coordinates[:, [2,1,0]],
  style={'linewidth': 1},
  xlim=(0, 512),
  ylim=(0, 512),
  title='',
)

### Reformat prediction outputs into array

In [ ]:
pred_s, pred_t = data.get_pred(output, labels, coordinates)

### Extract features and visualize tracking and segmentation

In [ ]:
n_tracks = features.get_tracks(labels)
print(n_tracks)

In [ ]:
contours = features.extract_contours(output) 

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=None, framenum=True, labelnum=False)
movie = visualize.show_animation(scene, title='', delay=500) 

In [ ]:
polygons =  features.polygons_per_cluster(labels, countours, output, include_noise=False)

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=None, framenum=True, labelnum=True)
cell_scene, label = visualize.select_cell(coordinates, labels, w=40, l=0) # choose image window (w) and label (l)
movie = visualize.show_animation(cell_scene, title='', delay=500)

In [ ]:
ax = visualize.plot_area_profile(polygons, label=0, ax=None, title=None)

In [ ]:
poly_area, growth_rate = features.get_area_and_growth_rate(polygons, labels, pred_s)